In [ ]:
import os
import fnmatch
import sys
sys.path.insert(0, '/opt/gpudb/api/python/gpudb/')
sys.path.insert(0, '/opt/gpudb/api/python/gpudb/packages')

import cStringIO
import collections
from avro import schema, io
import base64
import gpudb
import random
import math
from faker import Factory
fake = Factory.create()
import time

gpudb_host = "127.0.0.1"
my_collection = "MASTER"
my_table = "caltech256"
h_db = gpudb.GPUdb(encoding = 'BINARY', host = gpudb_host, port = '9191')

In [ ]:
h_db.clear_table( table_name = my_table, authorization = '', options = {} )

In [ ]:
x=[]
y=[]
response = h_db.get_records(table_name='netflow_sunman',offset=0,limit=20000)
res_decoded = gpudb.GPUdbRecord.decode_binary_data(response["type_schema"], response["records_binary"])
for index in range(19999):
    for k, v in res_decoded[index].items():
        if k == 'x':
            x.append(float(v))
        if k == 'y':
            y.append(float(v))
            
response = h_db.get_records(table_name='netflow_sunman',offset=20000,limit=20000)
res_decoded = gpudb.GPUdbRecord.decode_binary_data(response["type_schema"], response["records_binary"])
for index in range(19999):
    for k, v in res_decoded[index].items():
        if k == 'x':
            x.append(float(v))
        if k == 'y':
            y.append(float(v))
   

In [320]:
COUNT = 0
def getLatLong():
    global COUNT
    COUNT = COUNT+1
    return x[COUNT], y[COUNT]


In [324]:
columns = []
columns.append(gpudb.GPUdbRecordColumn("id", gpudb.GPUdbRecordColumn._ColumnType.STRING, [gpudb.GPUdbColumnProperty.CHAR16]))
columns.append(gpudb.GPUdbRecordColumn("state_province", gpudb.GPUdbRecordColumn._ColumnType.STRING, [gpudb.GPUdbColumnProperty.CHAR32]))
columns.append(gpudb.GPUdbRecordColumn("country", gpudb.GPUdbRecordColumn._ColumnType.STRING, [gpudb.GPUdbColumnProperty.CHAR64]))
columns.append(gpudb.GPUdbRecordColumn("x", gpudb.GPUdbRecordColumn._ColumnType.FLOAT))
columns.append(gpudb.GPUdbRecordColumn("y", gpudb.GPUdbRecordColumn._ColumnType.FLOAT))
columns.append(gpudb.GPUdbRecordColumn("date", gpudb.GPUdbRecordColumn._ColumnType.STRING, [gpudb.GPUdbColumnProperty.DATE]))
columns.append(gpudb.GPUdbRecordColumn("timestamp", gpudb.GPUdbRecordColumn._ColumnType.LONG, [gpudb.GPUdbColumnProperty.TIMESTAMP]))
columns.append(gpudb.GPUdbRecordColumn("image", gpudb.GPUdbRecordColumn._ColumnType.BYTES))

# Create the type object
image_record_type = gpudb.GPUdbRecordType(columns, label="image_record_type")
print image_record_type

""" Create the type in the database and save the type ID, needed to create
    a table in the next step """
image_record_type.create_type(h_db)
image_type_id = image_record_type.type_id



In [ ]:
response = h_db.create_table(table_name=my_table, type_id=image_type_id,options = {"collection_name":my_collection})
print "Table created:  {}".format(response['status_info']['status'])


In [321]:
encoded_obj_list = []


for root, dirs, files in os.walk("/home/jupyter/256_ObjectCategories"):
    for file in files:
        if file.endswith(".jpg"):
             with open(os.path.join(root, file), "rb") as imageFile:
                  str = imageFile.read()
                  lon, lat=getLatLong()
                  encoded_obj_list.append(gpudb.GPUdbRecord(image_record_type, [file, fake.state(), fake.country(), lon, lat, fake.date(), int(time.time()), str]).binary_data)



In [322]:
response = h_db.insert_records(table_name=my_table, data=encoded_obj_list, list_encoding="binary", options={})
print response

OrderedDict([(u'record_ids', []), (u'count_inserted', 30607), (u'count_updated', 0), ('status_info', OrderedDict([(u'status', u'OK'), (u'message', u''), (u'data_type', u'insert_records_response_avro'), ('response_time', 24.5383)]))])
